In [3]:
# Connecting to the database 
# Check connection:

import mysql.connector
import csv
import os

# Connect to the database
try:
    mydb = mysql.connector.connect(
        host="10.18.16.100",
        # username and password intentionally removed after testing 
        user="jmc",
        password="jmc",
        database="NDEX_RealTime_2018"
    )
    print(f"Successfully connected to the database: {mydb.database}")
except mysql.connector.Error as err:
    print(f"Error connecting to database: {err}")
    exit()

# Create a cursor object
mycursor = mydb.cursor()

# Close the database connection
mydb.close()

Successfully connected to the database: NDEX_RealTime_2018


In [30]:
# 50 sensors dataset is used based on 2018
# Extracting sensor data and visualising with a smaller dataset
import func_create_map
map = func_create_map.create_map('./detector/detectors2018_50_sensors.csv')
map

c:\Users\MSI\Documents\GitHub\I15_road_graph\func_create_map.py:20: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for loc, lanes in unique_locations.iteritems():


In [60]:
# Create Graph for 50 sensors
# This is a static distance based graph
import func_create_graph
data = pd.read_csv('./detector/detectors2018_50_sensors.csv')
graph = func_create_graph.create_graph(data)
graph

{0: [(1, 0.0751125136271385),
  (3, 0.4070866529804076),
  (2, 0.459868170280727),
  (4, 0.5794622137326149),
  (5, 0.7165860209301425),
  (6, 1.1335845692150728),
  (7, 1.3248347265689984),
  (8, 1.6397717724822098),
  (9, 1.9999232714143211)],
 1: [(0, 0.0751125136271385),
  (3, 0.342287479960648),
  (2, 0.38487408607758306),
  (4, 0.5114396510959607),
  (5, 0.6494023477163873),
  (6, 1.0660907330679665),
  (7, 1.2531851361962179),
  (8, 1.5724502701326604),
  (9, 1.9304882425616277)],
 2: [(4, 0.2644626390827099),
  (3, 0.23388589569924823),
  (5, 0.37915427794190104),
  (1, 0.38487408607758306),
  (0, 0.459868170280727),
  (6, 0.7596181497836048),
  (7, 0.9061094105838109),
  (8, 1.255880912204211),
  (9, 1.5948110015268833)],
 3: [(4, 0.17587485391383284),
  (2, 0.23388589569924823),
  (5, 0.31032639162712283),
  (1, 0.342287479960648),
  (0, 0.4070866529804076),
  (6, 0.7270575861361793),
  (7, 0.927755472842432),
  (8, 1.2329673885677812),
  (9, 1.5952597447225796)],
 4: [(5, 0.

In [82]:
import pandas as pd
from geopy.distance import distance
import numpy as np
from scipy.spatial import distance_matrix
from heapq import nsmallest
import datetime

# Set the number of closest node each node will be connected to:
num_connections = 9

# The nodes which will be ignored during graph making. Each nodes are represented as (Lat1, Long1)
ignore_list = [(36.068619, -115.211891),(35.759998,-115.760002),(35.650002,-115.650002), (41.2345, -112.6789), (42.3456, -113.7890)]

# Load data
data = pd.read_csv('./detector/detectors2018_50_sensors.csv')
latitude = data.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
longitude = data.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))
ids = data.iloc[:, 1]

locations, index = np.unique(np.column_stack((latitude, longitude)), axis=0, return_index=True)
ids = ids[index]

# Check if each location is in the ignore list and remove it if it is
mask = [tuple(loc) not in ignore_list for loc in locations]
locations = locations[mask]
ids = ids[mask]

# Calculate the distance matrix between all remaining locations
dist_matrix = distance_matrix(locations, locations)

tempgraph = {}

for i in range(len(locations)):
    tempgraph[i] = []

# Set the weight factor depending on the current time
current_time = datetime.datetime.now().time()
if current_time >= datetime.time(7, 0,0,0) and current_time <= datetime.time(9, 0,0,0):
    weight_factor = 1.5
    time_status = "Peak hours"
elif current_time >= datetime.time(16, 0,0,0) and current_time <= datetime.time(18, 0,0,0):
    weight_factor = 1.5
    time_status = "Peak hours"
elif current_time >= datetime.time(23, 0,0,0) or current_time <= datetime.time(6, 59,0,0):
    weight_factor = 0.5
    time_status = "Off-peak hours"
else:
    weight_factor = 1
    time_status = "Normal hours"

# Connect each node to the closest num_connections nodes
for i in range(len(locations)):
    closest = nsmallest(num_connections+1, range(len(dist_matrix[i])), key=lambda x: dist_matrix[i][x])
    closest.remove(i)
    for j in closest:
        distance_km = float(distance(locations[i], locations[j]).kilometers)* weight_factor
        tempgraph[i].append((j, distance_km))

print(f"Current time: {current_time}, Time status: {time_status}")

tempgraph

Current time: 09:57:30.897022, Time status: Normal hours


{0: [(1, 0.0751125136271385),
  (3, 0.4070866529804076),
  (2, 0.459868170280727),
  (4, 0.5794622137326149),
  (5, 0.7165860209301425),
  (6, 1.1335845692150728),
  (7, 1.3248347265689984),
  (8, 1.6397717724822098),
  (9, 1.9999232714143211)],
 1: [(0, 0.0751125136271385),
  (3, 0.342287479960648),
  (2, 0.38487408607758306),
  (4, 0.5114396510959607),
  (5, 0.6494023477163873),
  (6, 1.0660907330679665),
  (7, 1.2531851361962179),
  (8, 1.5724502701326604),
  (9, 1.9304882425616277)],
 2: [(4, 0.2644626390827099),
  (3, 0.23388589569924823),
  (5, 0.37915427794190104),
  (1, 0.38487408607758306),
  (0, 0.459868170280727),
  (6, 0.7596181497836048),
  (7, 0.9061094105838109),
  (8, 1.255880912204211),
  (9, 1.5948110015268833)],
 3: [(4, 0.17587485391383284),
  (2, 0.23388589569924823),
  (5, 0.31032639162712283),
  (1, 0.342287479960648),
  (0, 0.4070866529804076),
  (6, 0.7270575861361793),
  (7, 0.927755472842432),
  (8, 1.2329673885677812),
  (9, 1.5952597447225796)],
 4: [(5, 0.